# Counterfactuals guided by prototypes on Boston housing dataset

This notebook goes through an example of [prototypical counterfactuals](../methods/CFProto.ipynb) using [k-d trees](https://en.wikipedia.org/wiki/K-d_tree) to build the prototypes. Please check out [this notebook](./cfproto_mnist.ipynb) for a more in-depth application of the method on MNIST using (auto-)encoders and trust scores.

In this example, we will train a simple neural net to predict whether house prices in the Boston area are above the median value or not. We can then find a counterfactual to see which variables need to be changed to increase or decrease a house price above or below the median value.

In [2]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)  # suppress deprecation messages
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.datasets import load_boston
from alibi.explainers import CounterFactualProto
from alibi.explainers import CounterFactual

## Load and prepare Boston housing dataset

In [5]:
boston = load_boston()
data = boston.data
target = boston.target
feature_names = boston.feature_names
print(data)
print(target)

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]
[24.  21.6 34.7 33.4 36.2 28.7 22.9 27.1 16.5 18.9 15.  18.9 21.7 20.4
 18.2 19.9 23.1 17.5 20.2 18.2 13.6 19.6 15.2 14.5 15.6 13.9 16.6 14.8
 18.4 21.  12.7 14.5 13.2 13.1 13.5 18.9 20.  21.  24.7 30.8 34.9 26.6
 25.3 24.7 21.2 19.3 20.  16.6 14.4 19.4 19.7 20.5 25.  23.4 18.9 35.4
 24.7 31.6 23.3 19.6 18.7 16.  22.2 25.  33.  23.5 19.4 22.  17.4 20.9
 24.2 21.7 22.8 23.4 24.1 21.4 20.  20.8 21.2 20.3 28.  23.9 24.8 22.9
 23.9 26.6 22.5 22.2 23.6 28.7 22.6 22.  22.9 25.  20.6 28.4 21.4 38.7
 43.8 33.2 27.5 26.5 18.6 19.3 20.1 19.5 19.5 20.4 19.8 19.

Transform into classification task: target becomes whether house price is above the overall median or not

In [6]:
y = np.zeros((target.shape[0],))
y[np.where(target > np.median(target))[0]] = 1

Remove categorical feature

In [7]:
data = np.delete(data, 3, 1)
feature_names = np.delete(feature_names, 3)

Explanation of remaining features:

- CRIM: per capita crime rate by town
- ZN: proportion of residential land zoned for lots over 25,000 sq.ft.
- INDUS: proportion of non-retail business acres per town
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per USD10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

Standardize data

In [8]:
mu = data.mean(axis=0)
sigma = data.std(axis=0)
data = (data - mu) / sigma

Define train and test set

In [11]:
idx = 475
x_train,y_train = data[:idx,:], y[:idx]
x_test, y_test = data[idx:,:], y[idx:]
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(x_train[1])
print(y_train[1])

[-0.41733926 -0.48772236 -0.59338101 -0.74026221  0.19427445  0.36716642
  0.55715988 -0.8678825  -0.98732948 -0.30309415  0.44105193 -0.49243937]
[0. 1.]


## Train model

In [7]:
np.random.seed(0)
tf.set_random_seed(0)

In [8]:
def nn_model():
    x_in = Input(shape=(12,))
    x = Dense(40, activation='relu')(x_in)
    x = Dense(40, activation='relu')(x)
    x_out = Dense(2, activation='softmax')(x)
    nn = Model(inputs=x_in, outputs=x_out)
    nn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return nn

In [9]:
nn = nn_model()
nn.summary()
nn.fit(x_train, y_train, batch_size=64, epochs=500, verbose=0)
#nn.save('nn_boston.h5', save_format='h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 12)                0         
_________________________________________________________________
dense (Dense)                (None, 40)                520       
_________________________________________________________________
dense_1 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 82        
Total params: 2,242
Trainable params: 2,242
Non-trainable params: 0
_________________________________________________________________


In [18]:
score = nn.evaluate(x_test, y_test, verbose=0)
print('Test accuracy: ', score[1])
print(x_test[1].reshape((1,) + x_test[1].shape))
print('test: ', nn.predict(x_test[1].reshape((1,) + x_test[1].shape)))

Test accuracy:  0.83870965
[[ 0.14638431 -0.48772236  1.01599907  0.51229565  0.28402795  0.88990683
  -0.7081777   1.66124525  1.53092646  0.80657583  0.43348653  0.84481853]]
test:  [[0.937028   0.06297205]]


## Generate counterfactual guided by the nearest class prototype

Original instance:

In [11]:
X = x_test[1].reshape((1,) + x_test[1].shape)
shape = X.shape
print(shape)

(1, 12)


Run counterfactual:

In [12]:
# define model
#nn = load_model('nn_boston.h5')
predict_fn = lambda x: nn.predict(x)
# initialize explainer, fit and generate counterfactual
cf = CounterFactual(predict_fn, shape=shape, target_proba=1.0, tol=0.01,
                    target_class='other', max_iter=1000, lam_init=1e-1,
                    max_lam_steps=10, learning_rate_init=0.1)


#cf.fit(x_train)
explanation = cf.explain(X)
print(explanation)

dtype=float32), 'distance': 13.15241813659668, 'lambda': 0.000505, 'index': 58, 'class': 1, 'proba': array([[0.00333154, 0.9966685 ]], dtype=float32), 'loss': 0.006653069934032334}, {'X': array([[-0.97784096,  0.64816016,  2.1675205 , -0.6444657 , -0.81634307,
        -0.2926612 , -1.8896298 ,  2.8931873 ,  0.45237184,  0.32401267,
        -0.64815134, -0.38811615]], dtype=float32), 'distance': 13.140413284301758, 'lambda': 0.000505, 'index': 59, 'class': 1, 'proba': array([[0.00332155, 0.9966785 ]], dtype=float32), 'loss': 0.006646941259715263}, {'X': array([[-0.97729254,  0.6478734 ,  2.1665099 , -0.64214754, -0.8130952 ,
        -0.2922133 , -1.8892746 ,  2.8928287 ,  0.45313638,  0.32572302,
        -0.64660275, -0.38818476]], dtype=float32), 'distance': 13.127886772155762, 'lambda': 0.000505, 'index': 60, 'class': 1, 'proba': array([[0.00331194, 0.99668807]], dtype=float32), 'loss': 0.006640551714087193}, {'X': array([[-0.97670865,  0.6475523 ,  2.1654575 , -0.6397775 , -0.8097905

The prediction flipped from 0 (value below the median) to 1 (above the median):

In [13]:
print('Original prediction: {}'.format(explanation['orig_class']))
print('Counterfactual prediction: {}'.format(explanation['cf']['class']))

Original prediction: 0
Counterfactual prediction: 1


Let's take a look at the counterfactual. To make the results more interpretable, we will first undo the pre-processing step and then check where the counterfactual differs from the original instance:

In [14]:
orig = X * sigma + mu
counterfactual = explanation['cf']['X'] * sigma + mu
delta = counterfactual - orig
for i, f in enumerate(feature_names):
    if np.abs(delta[0][i]) > 1e-4:
        print('{}: {}'.format(f, delta[0][i]))

CRIM: -0.04421987328011312
ZN: 1.0019603565180013
INDUS: 0.06687389860129045
NOX: 0.0016002675807592626
RM: -0.002331675801022115
AGE: -0.7816179094894125
DIS: -0.040361733434935765
RAD: -0.02890238549689883
TAX: -3.9303862455470835
PTRATIO: 0.058080826355705995
B: -0.03003547929159822
LSTAT: -16.596271655271543


So in order to increase the house price, the proportion of owner-occupied units built prior to 1940 should decrease by ~11-12%. This is not surprising since the proportion for the observation is very high at 93.6%. Furthermore, the % of the population with "lower status" should decrease by ~5%.

In [16]:
print('% owner-occupied units built prior to 1940: {}'.format(orig[0][5]))
print('% lower status of the population: {}'.format(orig[0][11]))

% owner-occupied units built prior to 1940: 93.6
% lower status of the population: 18.68


Clean up:

In [16]:
os.remove('nn_boston.h5')